<a href="https://colab.research.google.com/github/sirius0830/capstone1/blob/master/snap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.request import urlretrieve
import os
import time
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
import pandas as pd
from selenium.webdriver.common import keys

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

# 현재 경로 확인
#print(os.getcwd())

#구글드라이브로 경로 설정
os.chdir("/content/drive/MyDrive")
print(os.getcwd())

In [ ]:
# 크롤링 시작 페이지
page_num = 1
# 마지막 페이지
last_page_num = 2

# 날짜
month = 1
date_list = []
link_list = []

while month <= 12:
  while page_num < last_page_num:
       
    url = "https://www.musinsa.com/mz/streetsnap?_y=2018&area=002&_mon={}&gender=&p={}#listStart".format(month, page_num)
    driver.get(url)
    driver.implicitly_wait(10)

    img_num = 1

    # 해당 사이트 url 정보 가져옴(html)
    html = driver.page_source
    # BeautifulSoup 파싱
    soup = BeautifulSoup(html,'html.parser')
    item_list = soup.find_all('span',{'class':'division'})
        
    #  날짜 리스트
    # for item in item_list:
    #   date_list.append(item)
    
    # 한 페이지에 60개
    while img_num <= 60:
      driver.find_element(By.CSS_SELECTOR,'#wrapper > div.bottom-column.column.clearfix > div.main-content-wrapper > div.section > div > div > div.list-box.box > ul > li:nth-child({}) > div.articleImg > a'.format(img_num)).send_keys(Keys.ENTER)
      img_url = driver.find_elements(By.CSS_SELECTOR, '.lbox')[0].get_attribute('href')
      
      # 해당 사이트 url 정보 가져옴(html)
      html = driver.page_source
      # BeautifulSoup 파싱
      soup = BeautifulSoup(html,'html.parser')

      link_list.append(img_url)


      try:
        # img_url에서 이미지 다운로드, snap 폴더에 '월-page_num-img_num.jpg' 형태로 저장
          urlretrieve(img_url, '{}/{}월_{}페이지_{}.jpg'.format('snap',month, page_num, img_num)) 
            
      except: # 오류 시 오류 선언하고 pass
          print('some error!(page num: {}, img num: {})'.format(page_num, img_num))
          pass
        
      # 뒤로가기 대신 url 재접속을 사용(오류 최소화)
      try:
        driver.get(url)
        driver.implicitly_wait(10)
        img_num += 1

      except TimeoutException as ex:
        print(ex.Message)
        driver.navigate().refresh()
            
    page_num += 1
  # 다음달 넘어가면서 1페이지로 초기화
  month += 1
  page_num = 1
  
# 날짜 엑셀 파일로 정리
# df = pd.DataFrame({'날짜': date_list})
# df.to_csv('날짜.csv', encoding='utf-8-sig')

# 링크 csv 파일로 정리
df = pd.DataFrame({'사진 링크': link_list})
df.to_csv('링크.csv', encoding='utf-8-sig')

some error!(page num: 1, img num: 52)
